# Deploy Watson ML

Deploys model to IBM Watson Machine Learning
Currently only supports TensorFlow V2.4 SavedModel format. More to come...

In [ ]:
!pip3 install ibm-watson-machine-learning

In [ ]:
from ibm_watson_machine_learning import APIClient
import json
import logging
import sys
import time
import re
import shutil
import os

In [ ]:
# IBM Cloud api_key https://cloud.ibm.com/iam/apikeys
api_key = os.environ.get('api_key')

# IBM Cloud deployment space ID https://dataplatform.cloud.ibm.com/ml-runtime/spaces
space = os.environ.get('space')

# IBM Cloud location (default: us-south)
location = os.environ.get('location', 'us-south')

# Model zip file name
model_zip = os.environ.get('model_zip', 'model.zip')

# IBM Cloud WML Deployment Name
deployment_name = os.environ.get('deployment_name', 'Deployment Name')

# IBM Cloud WML Model Name
model_name = os.environ.get('model_name', 'Model Name')

# temporal data storage for local execution
data_dir = os.environ.get('data_dir', '../../data/')

In [ ]:
parameters = list(
    map(lambda s: re.sub('$', '"', s),
        map(
            lambda s: s.replace('=', '="'),
            filter(
                lambda s: s.find('=') > -1 and bool(re.match(r'[A-Za-z0-9_]*=[.\/A-Za-z0-9]*', s)),
                sys.argv
            )
    )))

for parameter in parameters:
    logging.warning('Parameter: ' + parameter)
    exec(parameter)

In [ ]:
wml_credentials = {
    "apikey": api_key,
    "url": 'https://' + location + '.ml.cloud.ibm.com'
}

In [ ]:
client = APIClient(wml_credentials)

In [ ]:
model_folder = str(time.time())
shutil.unpack_archive(data_dir + model_zip, extract_dir=data_dir + model_folder)
data_dir_model_folder = data_dir + model_folder

In [ ]:
%%bash -s "$data_dir_model_folder"
cd $1
tar -czvf ../model.tar.gz *
cd ..

In [ ]:
o = client.software_specifications.get_uid_by_name('tensorflow_2.4-py3.7')
software_spec_uid = o
client.set.default_space(space)

In [ ]:
model_meta_props = {
    client.repository.ModelMetaNames.NAME: deployment_name,
    client.repository.ModelMetaNames.TYPE: "tensorflow_2.4",
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: software_spec_uid
}

published_model = client.repository.store_model(
    model='model.tar.gz',
    meta_props=model_meta_props
)

model_uid = client.repository.get_model_uid(published_model)

In [ ]:
model_details = client.repository.get_details(model_uid)
print(json.dumps(model_details, indent=2))

In [ ]:
client.repository.list_models(limit=15)

In [ ]:
deployment = client.deployments.create(
    model_uid,
    meta_props={
        client.deployments.ConfigurationMetaNames.NAME: model_name,
        client.deployments.ConfigurationMetaNames.ONLINE: {}
    }
)

deployment_uid = client.deployments.get_uid(deployment)

In [ ]:
deployment_uid